In [1]:
from models.DAX.DAX_models import DAX_quantile_regression, DAX_baseline
from models.energy import energy_models
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
import numpy as np

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-03-24,DAX,1 day,-2.478520,-0.683375,-0.069389,0.490104,2.520942
1,2024-03-24,DAX,2 day,-3.349019,-0.945968,-0.105510,0.624821,3.593514
2,2024-03-24,DAX,3 day,-4.027670,-1.209531,-0.195276,0.774530,4.315379
3,2024-03-24,DAX,4 day,-4.763757,-1.437087,-0.246887,0.844792,5.009079
4,2024-03-24,DAX,5 day,-5.019355,-1.552304,-0.377205,0.887736,5.062647


In [6]:
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [7]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,DAX,1 day,-2.659740,-0.500982,0.077228,0.710654,2.719772
1,2024-02-14,DAX,2 day,-3.746485,-0.727760,0.103141,0.978977,3.570492
2,2024-02-14,DAX,5 day,-4.317083,-0.830364,0.178931,1.251251,4.628697
3,2024-02-14,DAX,6 day,-5.058731,-0.898230,0.218584,1.451656,5.367396
4,2024-02-14,DAX,7 day,-5.321097,-0.946094,0.346121,1.607134,6.259934


In [8]:
quantile

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,DAX,1 day,-2.697315,-0.567931,0.122281,0.658111,1.895334
1,2024-02-14,DAX,2 day,-4.685462,-2.601772,-1.799531,-1.210189,0.328829
2,2024-02-14,DAX,5 day,-3.551711,-1.552685,-0.613763,-0.011930,1.523145
3,2024-02-14,DAX,6 day,-4.022736,-2.004049,-1.175835,-0.481863,1.149809
4,2024-02-14,DAX,7 day,-3.488536,-1.683901,-0.756083,-0.046446,1.638068


In [9]:
df_dax = naive_ensemble.ensemble(base,quantile,0.8,0.2)

In [10]:
df_dax

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,DAX,1 day,-2.667255,-0.514372,0.086239,0.700145,2.554884
1,2024-02-14,DAX,2 day,-3.934280,-1.102562,-0.277393,0.541144,2.922160
2,2024-02-14,DAX,5 day,-4.164009,-0.974828,0.020392,0.998615,4.007587
3,2024-02-14,DAX,6 day,-4.851532,-1.119393,-0.060300,1.064952,4.523879
4,2024-02-14,DAX,7 day,-4.954584,-1.093656,0.125680,1.276418,5.335561


In [11]:
energydata= get_energy.get()

  0%|          | 0/269 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 269/269 [00:30<00:00,  8.76it/s]


In [24]:
df_energy= energy_models.energy_XGBoost(energydata,date_str='2024-02-14')

In [25]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,energy,36 hour,64.045959,68.483833,71.047020,73.305389,76.355835
1,2024-02-14,energy,40 hour,59.859379,63.551311,66.540077,68.225845,71.914993
2,2024-02-14,energy,44 hour,60.065620,62.567818,65.188622,66.502022,67.780556
3,2024-02-14,energy,60 hour,57.069286,59.153980,60.861267,63.005474,63.766880
4,2024-02-14,energy,64 hour,52.906685,54.493233,55.423454,58.016098,61.773037
5,2024-02-14,energy,68 hour,53.354237,56.130596,56.151104,57.989506,59.095627


In [26]:
#Reorder quantiles to prevent quantile crossing

In [27]:
df = merge_submissions.merge(df_energy,df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2024-02-14         DAX    1 day  -2.667255  -0.514372   0.086239   
1    2024-02-14         DAX    2 day   -3.93428  -1.102562  -0.277393   
2    2024-02-14         DAX    5 day  -4.164009  -0.974828   0.020392   
3    2024-02-14         DAX    6 day  -4.851532  -1.119393    -0.0603   
4    2024-02-14         DAX    7 day  -4.954584  -1.093656    0.12568   
0    2024-02-14      energy  36 hour  64.045959  68.483833   71.04702   
1    2024-02-14      energy  40 hour  59.859379  63.551311  66.540077   
2    2024-02-14      energy  44 hour   60.06562  62.567818  65.188622   
3    2024-02-14      energy  60 hour  57.069286   59.15398  60.861267   
4    2024-02-14      energy  64 hour  52.906685  54.493233  55.423454   
5    2024-02-14      energy  68 hour  53.354237  56.130596  56.151104   
0    2024-02-14  infections   0 week         NA         NA         NA   
1    2024-02-14  infections   1 week         NA    

In [28]:
import pandas as pd

In [29]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [30]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,DAX,1 day,-2.667255,-0.514372,0.086239,0.700145,2.554884
1,2024-02-14,DAX,2 day,-3.93428,-1.102562,-0.277393,0.541144,2.92216
2,2024-02-14,DAX,5 day,-4.164009,-0.974828,0.020392,0.998615,4.007587
3,2024-02-14,DAX,6 day,-4.851532,-1.119393,-0.0603,1.064952,4.523879
4,2024-02-14,DAX,7 day,-4.954584,-1.093656,0.12568,1.276418,5.335561
0,2024-02-14,energy,36 hour,64.045959,68.483833,71.04702,73.305389,76.355835
1,2024-02-14,energy,40 hour,59.859379,63.551311,66.540077,68.225845,71.914993
2,2024-02-14,energy,44 hour,60.06562,62.567818,65.188622,66.502022,67.780556
3,2024-02-14,energy,60 hour,57.069286,59.15398,60.861267,63.005474,63.76688
4,2024-02-14,energy,64 hour,52.906685,54.493233,55.423454,58.016098,61.773037


In [31]:
df.to_csv('../../forecasts/20240214_JonSnow.csv',index=False)

In [32]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-02-14,DAX,1 day,-2.667255,-0.514372,0.086239,0.700145,2.554884
1,2024-02-14,DAX,2 day,-3.93428,-1.102562,-0.277393,0.541144,2.92216
2,2024-02-14,DAX,5 day,-4.164009,-0.974828,0.020392,0.998615,4.007587
3,2024-02-14,DAX,6 day,-4.851532,-1.119393,-0.0603,1.064952,4.523879
4,2024-02-14,DAX,7 day,-4.954584,-1.093656,0.12568,1.276418,5.335561
0,2024-02-14,energy,36 hour,64.045959,68.483833,71.04702,73.305389,76.355835
1,2024-02-14,energy,40 hour,59.859379,63.551311,66.540077,68.225845,71.914993
2,2024-02-14,energy,44 hour,60.06562,62.567818,65.188622,66.502022,67.780556
3,2024-02-14,energy,60 hour,57.069286,59.15398,60.861267,63.005474,63.76688
4,2024-02-14,energy,64 hour,52.906685,54.493233,55.423454,58.016098,61.773037
